# Advance Regex For Data Mining

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices. **This Series should be sorted by a tie-break sort in the format of ("extracted date", "original row number").**

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [1]:
import pandas as pd
import re

doc = []
with open('assets/dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [2]:
mydf=df.copy()
mydf

0           03/25/93 Total time of visit (in minutes):\n
1                         6/18/85 Primary Care Doctor:\n
2      sshe plans to move as of 7/8/71 In-Home Servic...
3                  7 on 9/27/75 Audit C Score Current:\n
4      2/6/96 sleep studyPain Treatment Pain Level (N...
                             ...                        
495    1979 Family Psych History: Family History of S...
496    therapist and friend died in ~2006 Parental/Ca...
497                         2008 partial thyroidectomy\n
498    sPt describes a history of sexual abuse as a c...
499    . In 1980, patient was living in Naples and de...
Length: 500, dtype: object

\d{1,2}[\/-]\d{1,2}[\/-]\d{2,4}|(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)+[.]?[\s-]?\d{1,2}[\-,\s][\s]?\d{2,4}|\d{1,2}[\s](?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)+[.,\s]?[\s]?\d{2,4}|(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)+[\s]?\w+[,]?[\s]?\d{2,4}|\d{1,2}[\/-]\d{2,4}|\d{4}

In [2]:
"""(?P<date>(?P<month1>\d{1,2})[\/-](?P<day1>\d{1,2})[\/-](?<year1>\d{2,4})|(?<month2>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)+)[.]?[\s-]?(?P<day2>\d{1,2})[\-,\s][\s]?(?P<year2>)\d{2,4}|(?P<day3>\d{1,2})[\s](?P<month3>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)+)[.,\s]?[\s]?(?P<year3>\d{2,4})|(?P<month4>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)+)[\s]?\w+[,]?[\s]?(?P<year4>\d{2,4})|(?P<day5>\d{1,2})[\/-](?P<year>\d{2,4})|(?P<year6>\d{4}))"""

'(?P<date>(?P<month1>\\d{1,2})[\\/-](?P<day1>\\d{1,2})[\\/-](?<year1>\\d{2,4})|(?<month2>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)+)[.]?[\\s-]?(?P<day2>\\d{1,2})[\\-,\\s][\\s]?(?P<year2>)\\d{2,4}|(?P<day3>\\d{1,2})[\\s](?P<month3>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)+)[.,\\s]?[\\s]?(?P<year3>\\d{2,4})|(?P<month4>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)+)[\\s]?\\w+[,]?[\\s]?(?P<year4>\\d{2,4})|(?P<day5>\\d{1,2})[\\/-](?P<year>\\d{2,4})|(?P<year6>\\d{4}))'

Pattern under Development:
(?P<month1>\d{1,2})[\/-](?P<day1>\d{1,2})[\/-](?P<year1>\d{2,4})|(?P<month2>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)+)[.]?[\s-]?(?P<day2>\d{1,2})[\-,\s][\s]?(?P<year2>\d{2,4})|(?P<day3>\d{1,2})[\s](?P<month3>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)+)[.,\s]?[\s]?(?P<year3>\d{2,4})|(?P<month4>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[A-Za-z]*)[\s]?[,]?[\s]?(?P<year4>\d{2,4})|(?P<day5>\d{1,2})[\/](?P<year5>\d{2,4})|\b[a-zA-Z]?(?P<year6>19[0-9]{2}|20[0-1][0-9]|20[0-2][0-4]\b)

In [ ]:
r'\d{1,2}[\/-]\d{1,2}[\/-]\d{2,4}|(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)+[.]?[\s-]?\d{1,2}[\-,\s][\s]?\d{2,4}|'\
    r'\d{1,2}[\s](?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)+[.,\s]?[\s]?\d{2,4}|'\
        r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[A-Za-z]*[\s]?[,]?[\s]?\d{2,4}|'\
            r'\d{1,2}[\/]\d{2,4}|'\
                r'[^\-][\s]?\d{4}[\s,][^\-]'

In [15]:
df.str.findall(r'\d{1,2}[\/-]\d{1,2}[\/-]\d{2,4}|(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)+[.]?[\s-]?\d{1,2}[\-,\s][\s]?\d{2,4}|\d{1,2}[\s](?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)+[.,\s]?[\s]?\d{2,4}|(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[A-Za-z]*[\s]?[,]?[\s]?\d{2,4}|\d{1,2}[\/]\d{2,4}|[^\-][\s]?\d{4}[\s,][^\-]')[485]

[]

In [3]:
df.str.findall(r'(?P<month1>\d{1,2})[\/-](?P<day1>\d{1,2})[\/-](?P<year1>\d{2,4})|(?P<month2>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[A-Za-z]*)[.]?[\s-]?(?P<day2>\d{1,2})[\-,\s][\s]?(?P<year2>\d{2,4})|(?P<day3>\d{1,2})[\s](?P<month3>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[A-Za-z]*)[.,\s]?[\s]?(?P<year3>\d{2,4})|(?P<month4>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[A-Za-z]*)[\s]?[,]?[\s]?(?P<year4>\d{2,4})|(?P<month5>\d{1,2})[\/](?P<year5>\d{2,4})|\b[a-zA-Z]?(?P<year6>19[0-9]{2}|20[0-1][0-9]|20[0-2][0-4]\b)')

0      [(03, 25, 93, , , , , , , , , , , )]
1       [(6, 18, 85, , , , , , , , , , , )]
2        [(7, 8, 71, , , , , , , , , , , )]
3       [(9, 27, 75, , , , , , , , , , , )]
4        [(2, 6, 96, , , , , , , , , , , )]
                       ...                 
495      [(, , , , , , , , , , , , , 1979)]
496      [(, , , , , , , , , , , , , 2006)]
497      [(, , , , , , , , , , , , , 2008)]
498      [(, , , , , , , , , , , , , 2005)]
499      [(, , , , , , , , , , , , , 1980)]
Length: 500, dtype: object

In [18]:
df.str.extract(r'(?P<month1>\d{1,2})[\/-](?P<day1>\d{1,2})[\/-](?P<year1>\d{2,4})|(?P<month2>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)+)[.]?[\s-]?(?P<day2>\d{1,2})[\-,\s][\s]?(?P<year2>)\d{2,4}|(?P<day3>\d{1,2})[\s](?P<month3>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)+)[.,\s]?[\s]?(?P<year3>\d{2,4})|(?P<month4>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[A-Za-z]*)[\s]?[,]?[\s]?(?P<year4>\d{2,4})|(?P<day5>\d{1,2})[\/-](?P<year>\d{2,4})|[^\-][ ]?(?P<year6>\d{4})[\s,][^\-]')[190:200]

,month1,day1,year1,month2,day2,year2,day3,month3,year3,month4,year4,day5,year,year6
190,NaN,NaN,NaN,NaN,NaN,NaN,10,Aug,2000,NaN,NaN,NaN,NaN,NaN
191,NaN,NaN,NaN,NaN,NaN,NaN,30,Nov,1972,NaN,NaN,NaN,NaN,NaN
192,NaN,NaN,NaN,NaN,NaN,NaN,06,May,1993,NaN,NaN,NaN,NaN,NaN
193,NaN,NaN,NaN,NaN,NaN,NaN,18,Jan,1995,NaN,NaN,NaN,NaN,NaN
194,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,April,11,NaN,NaN,NaN
195,NaN,NaN,NaN,May,30,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
196,NaN,NaN,NaN,Feb,18,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
197,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,February,18,NaN,NaN,NaN
198,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013
199,NaN,NaN,NaN,Jan,24,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
extracted_df=mydf.str.extract(r'(?P<month1>\d{1,2})[\/-](?P<day1>\d{1,2})[\/-](?P<year1>\d{2,4})|(?P<month2>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[A-Za-z]*)[.]?[\s-]?(?P<day2>\d{1,2})[\-,\s][\s]?(?P<year2>\d{2,4})|(?P<day3>\d{1,2})[\s](?P<month3>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[A-Za-z]*)[.,\s]?[\s]?(?P<year3>\d{2,4})|(?P<month4>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[A-Za-z]*)[\s]?[,]?[\s]?(?P<year4>\d{2,4})|(?P<month5>\d{1,2})[\/](?P<year5>\d{2,4})|\b[a-zA-Z]?(?P<year6>19[0-9]{2}|20[0-1][0-9]|20[0-2][0-4]\b)')
extracted_df

,month1,day1,year1,month2,day2,year2,day3,month3,year3,month4,year4,month5,year5,year6
0,03,25,93,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6,18,85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7,8,71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9,27,75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2,6,96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1979
496,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006
497,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008
498,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005


#Final
""""((?P<month>\d{1,2})[\/-](?P<day>\d{1,2})[\/-](?P<year>\d{2,4}))|((?P<month>[A-Za-z]+)[.]?[\s-]?(?P<day>\d{1,2})[\-,\s][\s]?(?P<year>\d{2,4}))|((?P<day>\d{1,2})[\s](?P<month>[A-Za-z]+)[.,\s]?[\s]?(?P<year>\d{2,4}))|((?P<month>[A-Za-z]+)[\s]?(?P<day>\w+)[,]?[\s]?(?P<year>\d{2,4}))|((?P<day>\d{1,2})[\/-](?P<year>\d{2,4}))|((?P<year>\d{4}))""""

"(?P<month>\d{1,2})[\/-](?P<day>\d{1,2})[\/-](?P<year>\d{2,4})|(?P<month2>[A-Za-z]+)[.]?[\s-]?(?P<day2>\d{1,2})[\-,\s][\s]?(?P<year2>\d{2,4})|(?P<day3>\d{1,2})[\s](?P<month3>[A-Za-z]+)[.,\s]?[\s]?(?P<year3>\d{2,4})|(?P<month4>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)+)[\s]?(?P<day4>\w+)[,]?[\s]?(?P<year4>\d{2,4})|(?P<day5>\d{1,2})[\/-](?P<year5>\d{2,4})|(?P<year6>\d{4})"

(?P<month1>\d{1,2})[\/-](?P<day1>\d{1,2})[\/-](?P<year1>\d{2,4})|(?P<month2>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)+)[.]?[\s-]?(?P<day2>\d{1,2})[\-,\s][\s]?(?P<year2>\d{2,4})|(?P<day3>\d{1,2})[\s](?P<month3>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)+)[.,\s]?[\s]?(?P<year3>\d{2,4})|(?P<month4>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[A-Za-z]*)[\s]?[,]?[\s]?(?P<year4>\d{2,4})|(?P<day5>\d{1,2})[\/](?P<year5>\d{2,4})|(?P<year6>\b(19[0-9]{2}|20[0-1][0-9]|20[0-2][0-4])\b)

In [ ]:
pattern=r'(?P<date>(?P<month1>\d{1,2})[\/-](?P<day1>\d{1,2})[\/-](?<year1>\d{2,4})|' \
    r'(?<month2>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)+)[.]?[\s-]?(?P<day2>\d{1,2})[\-,\s][\s]?(?P<year2>)\d{2,4}|' \
    r'(?P<day3>\d{1,2})[\s](?P<month3>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)+)[.,\s]?[\s]?(?P<year3>\d{2,4})|' \
    r'(?P<month4>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)+)[\s]?\w+[,]?[\s]?(?P<year4>\d{2,4})|' \
    r'(?P<day5>\d{1,2})[\/-](?P<year>\d{2,4})|' \
    r'(?P<year6>\d{4}))'

In [6]:
pattern=r'(?P<month1>\d{1,2})[\/-](?P<day1>\d{1,2})[\/-](?P<year1>\d{2,4})|' \
          r'(?P<month2>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)+)[.]?[\s-]?(?P<day2>\d{1,2})[\-,\s][\s]?(?P<year2>\d{2,4})|' \
          r'(?P<day3>\d{1,2})[\s](?P<month3>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)+)[.,\s]?[\s]?(?P<year3>\d{2,4})|' \
          r'(?P<month4>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)+)[\s]?(?P<day4>\d{1,2})[,]?[\s]?(?P<year4>\d{2,4})|' \
          r'(?P<day5>\d{1,2})[\/-](?P<year5>\d{2,4})|' \
          r'(?P<year6>\d{4})'

In [7]:
df=df.str.extract(pattern)
df

AttributeError: 'DataFrame' object has no attribute 'str'

In [6]:
extracted_df['day'] = extracted_df['day1'].fillna(extracted_df['day2']).fillna(extracted_df['day3'])
extracted_df['month'] = extracted_df['month1'].fillna(extracted_df['month2']).fillna(extracted_df['month3']).fillna(extracted_df['month4']).fillna(extracted_df['month5'])
extracted_df['year'] = extracted_df['year1'].fillna(extracted_df['year2']).fillna(extracted_df['year3']).fillna(extracted_df['year4']).fillna(extracted_df['year5']).fillna(extracted_df['year6'])
dates_df=extracted_df[['month','day','year']]
dates_df

,month,day,year
0,03,25,93
1,6,18,85
2,7,8,71
3,9,27,75
4,2,6,96
...,...,...,...
495,NaN,NaN,1979
496,NaN,NaN,2006
497,NaN,NaN,2008
498,NaN,NaN,2005


""""* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.
With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the
 original Series' indices.
 **This Series should be sorted by a tie-break sort in the format of ("extracted date", "original row number").**"""

In [ ]:
month_mapping = {
    '01': '01', 'Jan': '01', 'January': '01',
    '02': '02', 'Feb': '02', 'February': '02',
    '03': '03', 'Mar': '03', 'March': '03',
    '04': '04', 'Apr': '04', 'April': '04',
    '05': '05', 'May': '05',
    '06': '06', 'Jun': '06', 'June': '06',
    '07': '07', 'Jul': '07', 'July': '07',
    '08': '08', 'Aug': '08', 'August': '08',
    '09': '09', 'Sep': '09', 'September': '09',
    '10': '10', 'Oct': '10', 'October': '10',
    '11': '11', 'Nov': '11', 'November': '11',
    '12': '12', 'Dec': '12', 'December': '12',
}

In [7]:
typo_mapping = {
    'Janaury': '01',  # Typo for January
    'Decemeber': '12',  # Typo for December
}

In [9]:
dates_df.loc[:, 'day']=dates_df['day'].fillna(1)
dates_df.loc[:, 'month']=dates_df['month'].fillna(1).replace(typo_mapping)
dates_df.loc[:, 'year']=dates_df['year'].astype(int).apply(lambda x:1900+x if x<100 else x)

dates_data=dates_df[['month','day','year']]
dates_data

,month,day,year
0,03,25,1993
1,6,18,1985
2,7,8,1971
3,9,27,1975
4,2,6,1996
...,...,...,...
495,1,1,1979
496,1,1,2006
497,1,1,2008
498,1,1,2005


In [10]:
dates_data.isnull().sum()

month    0
day      0
year     0
dtype: int64

In [11]:
dates_data['month'] = pd.to_datetime(dates_data['month'], format='%m', errors='coerce').dt.strftime('%m').fillna(
    pd.to_datetime(dates_data['month'], format='%b', errors='coerce').dt.strftime('%m').fillna(
        pd.to_datetime(dates_data['month'], format='%B', errors='coerce').dt.strftime('%m')
    )
)
dates_data['date'] = dates_data['month'].astype(str) + '/' + dates_data['day'].astype(str) + '/' + dates_data['year'].astype(str)
dates_data

,month,day,year,date
0,03,25,1993,03/25/1993
1,06,18,1985,06/18/1985
2,07,8,1971,07/8/1971
3,09,27,1975,09/27/1975
4,02,6,1996,02/6/1996
...,...,...,...,...
495,01,1,1979,01/1/1979
496,01,1,2006,01/1/2006
497,01,1,2008,01/1/2008
498,01,1,2005,01/1/2005


In [13]:
dates_data['date'] = pd.to_datetime(dates_data['date'], format='%m/%d/%Y', errors='coerce')
dates_data

,month,day,year,date
0,03,25,1993,1993-03-25
1,06,18,1985,1985-06-18
2,07,8,1971,1971-07-08
3,09,27,1975,1975-09-27
4,02,6,1996,1996-02-06
...,...,...,...,...
495,01,1,1979,1979-01-01
496,01,1,2006,2006-01-01
497,01,1,2008,2008-01-01
498,01,1,2005,2005-01-01


In [25]:
sorted_df=dates_data.sort_values(by='date',kind='stable').index
sorted_df

Index([  9,  84,   2,  53,  28, 474, 153,  13, 129,  98,
       ...
       152, 235, 464, 427, 413, 253, 231, 141, 186, 161],
      dtype='int64', length=500)

In [16]:
years=pd.Series(sorted_df['year']).astype(int)
sorted_years=years.sort_values(,kind='stable').index
years_sorted=pd.Series(sorted_years)

In [14]:
def date_sorter(dates_data):

    order=dates_data.sort_values(by='date',kind='stable',ascending=True).index
    return pd.Series(order) # Your answer here

In [19]:
len(date_sorter(dates_data))

500

In [ ]:
# Function BLock to Check Working or Regex Pattern

In [20]:
import re
import numpy as np
s_test = date_sorter(dates_data)

def run_df_modified_check():
    """
    Check if df appears to be modified.
    """
    try:
        assert type(df) == pd.Series
        assert (df.index == pd.RangeIndex(start=0, stop=500, step=1)).all()
        assert (df.apply(type) == str).all()
        assert df.str.len().min() >= 6
        assert df.str[5].apply(ord).sum() == 38354
        print("Passed df modification check")
    except:
        print("Failed df modification check")

run_df_modified_check()

# check if running the code twice produces the same result
try:
    assert (date_sorter() == s_test).all()
    print("Passed repeatability check")
except:
    print("Failed repeatability check")

# check if the result has the expected index
try:
    # assert type(date_sorter().index) == pd.RangeIndex
    # assert (date_sorter().index == pd.RangeIndex(start=0, stop=500, step=1)).all()
    assert list(date_sorter().index) == list(range(500))
    print("Passed index check")
except:
    print("Failed index check")

# check the tie-break sort for a sample of records where some have the same date
# note that this only tests a sample and does not check the entire answer
try:
    test_indices = [335, 415, 323, 405, 370, 382, 303, 488, 283,
                    395, 318, 369, 493, 252, 314, 410, 490]
    answer_lkp = {original_index:answer_index for
                  answer_index, original_index in s_test.to_dict().items()}
    i_test = [answer_lkp[i] for i in test_indices]
    assert sorted(i_test) == i_test
    print("Passed secondary sort sample check")
except:
    print("Failed secondary sort sample check")

def run_v_check(s_test):
    """
    Check if the parsed dates appear to be correct and correctly sorted.
    The check works by producing some test checksums
    if you get for example a False entry in the agree column for
    index value 20 that would mean you have at least one incorrectly
    parsed or incorrectly sorted date in the **output** index
    range 20,21,...,29
    The results of the test are printed.
    Args:
    s_test: Series such as produced by date_sorter()
    Returns:
    None
    """
    try:
        v_check = pd.DataFrame({'correct':
        [6695, 14428, 16742, 9275, 12290, 14654, 9421, 10185, 11464, 16491,
         11797, 14036, 15459, 9412, 13069, 10400, 10498, 14322, 13274, 11001,
         11383, 11910, 10977, 9692, 10199, 10187, 15456, 13491, 9186, 13646,
         11142, 13724, 10994, 12905, 15968, 16648, 13966, 14607, 16932, 14622,
         17942, 18220, 17818, 18305, 19633, 12522, 13978, 18445, 20156, 14797],
        'learner':[
        (s_test.iloc[10*i:(i+1)*10].values * np.array(range(1,11))).sum() for i in range(50)]},
        index=range(0,500,10)).assign(agree=lambda x:x['correct']==x['learner'])
        print("Values checksums:")
        print(v_check)
        assert v_check['agree'].all()
        print("Passed values check")
    except:
        print("Failed values check")
    return

run_v_check(s_test)

Passed df modification check
Failed repeatability check
Failed index check
Passed secondary sort sample check
Values checksums:
     correct  learner  agree
0       6695     6695   True
10     14428    14428   True
20     16742    16742   True
30      9275     9275   True
40     12290    12290   True
50     14654    14654   True
60      9421     9421   True
70     10185    10185   True
80     11464    11464   True
90     16491    16491   True
100    11797    11797   True
110    14036    14036   True
120    15459    15459   True
130     9412     9412   True
140    13069    13069   True
150    10400    10400   True
160    10498    10498   True
170    14322    14322   True
180    13274    13274   True
190    11001    11001   True
200    11383    11383   True
210    11910    11910   True
220    10977    10977   True
230     9692     9692   True
240    10199    10199   True
250    10187    10187   True
260    15456    15456   True
270    13491    13491   True
280     9186     9186   True
29